## Логистическая регрессия

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import seaborn as sns
sns.set_theme(style='dark', font_scale=1.3)

import warnings
warnings.filterwarnings('ignore')

### 1.  Cкачаем [датасет](https://www.kaggle.com/colearninglounge/predicting-pulsar-starintermediate) о предсказании пульсаров.

Пульсары &mdash; это космические объекты, излучающие в различных диапазонах длины волны. Согласно современным астрофизическим теориям, пульсары представляют собой вращающиеся нейтронные звезды, обладающие магнитным полем, наклоненным относительно оси вращения.

Пульсары представляют значительный научный интерес. Поэтому есть потребность в автоматической идентификации интересных объектов для ускорения процесса изучения. На практике почти все обнаружения вызваны радиочастотными помехами и шумом, что затрудняет поиск достоверных сигналов. В последнее время для автоматического обнаружения пульсаров начали применять машинное обучение. Актуальная задача сегодня &mdash; бинарная классификация объектов-кандидатов на предмет того, являются ли они пульсарами.

В используемом датасете есть как примеры ложных обнаружений, так и примеры реальных пульсаров, подтвержденные учеными. Данные получены в результате [The High Time Resolution Universe Pulsar Survey I](https://arxiv.org/abs/1006.5744).

Мы будем работать только с файлом `pulsar_data_train.csv`.

In [ ]:
data = pd.read_csv("pulsar_data_train.csv")

Сигнал от пульсаров представляет собой периодические импульсы.

Для описания объекта используется integrated profile (интегральный профиль) &mdash; агрегирование сигнала за разные периоды вращения (<a target="_blank" href="http://ipta.phys.wvu.edu/files/student-week-2017/IPTA2017_KuoLiu_pulsartiming.pdf">ссылка</a>, слайд 11). У всех пульсаров разные интегральные профили, и обычно эта характеристика мало меняется во времени.
В данном датасете интегральный профиль также усреднен по частоте.

Кроме того, импульс приходит в разное время на разных частотах. Задержка от частоты к частоте вызвана наличием ионизированной межзвездной среды и называется дисперсией. Не путать с дисперсией в теории вероятностей.

С дисперсией связана еще одна характеристика объекта &mdash; DM-SNR кривая. Подробнее о ней можно почитать <a target="_blank" href="https://arxiv.org/pdf/1005.5068.pdf#:~:text=period-DM%20diagram%3B%20good%20agreement,pulsars%20in%20their%20candidate%20plots.)">в специализированных материалах</a>.

Обе характеристики, integrated profile и DM-SNR кривая, представляют собой одномерные функции.
При создании датасета значения функций были посчитаны в конечном количестве $n$ точек.
Фактически, были получены реализации выборки двух случайных величин.

Пусть $P = (p_1, \ldots p_n)$ &mdash; массив значений integrated profile.
Для него можно посчитать следующие величины:

1.   Выборочное среднее $\bar p = \frac{1}{n}\sum \limits_{i=1}^n p_i$;
2.   Выборочное стандартное отклонение $\sqrt{\frac{1}{n}\sum \limits_{i=1}^n (p_i - \bar p)^2}$;
3. Выборочный коэффициент асимметрии (skewness);
4. Выборочный коэффициент эксцесса (kurtosis).

Для $D = (d_1, \ldots d_n)$, массива значений DM-SNR, аналогично.

Именно эти значения по массивам $P$ и $D$ являются признаками в данном датасете.

Про коэффициенты эксцесса и асимметрии можно посмотреть <a target="_blank" href="https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D1%8D%D1%84%D1%84%D0%B8%D1%86%D0%B8%D0%B5%D0%BD%D1%82_%D1%8D%D0%BA%D1%81%D1%86%D0%B5%D1%81%D1%81%D0%B0">здесь</a> и <a target="_blank" href="https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D1%8D%D1%84%D1%84%D0%B8%D1%86%D0%B8%D0%B5%D0%BD%D1%82_%D0%B0%D1%81%D0%B8%D0%BC%D0%BC%D0%B5%D1%82%D1%80%D0%B8%D0%B8">здесь</a>, формулы для выборочных коэффициентов асимметрии и эксцесса можно найти в английской версии этих статей.

<a target="_blank" href="https://arxiv.org/pdf/1603.05166.pdf">Статья</a> с подробным описанием процесса генерации данных.










In [ ]:
data.head()

In [ ]:
data.info()

### 2. Предобработаем данные.

В этом датасете в некоторых столбцах есть пропуски в данных. Об этом говорит то, что значение Non-Null Count в таблице выше не равно количеству строк для некоторых столбцов. Чтобы не усложнять себе жизнь, пока просто не будем брать эти столбцы для анализа.

Посчитайем число пропусков в каждой колонке:

In [ ]:
data.isna().sum()

Удалите все соответствующие строчки:

In [ ]:
df = data.dropna()

In [ ]:
df.isna().sum()

Теперь возьмем только некоторые столбцы в качестве независимых переменных и выделим зависимую. Столбец `target_class` отвечает за целевую переменную — индикатор того, является ли объект пульсаром.

In [ ]:
needed_columns = [' Mean of the integrated profile',
       ' Standard deviation of the integrated profile',
       ' Skewness of the integrated profile', ' Mean of the DM-SNR curve',
       ' Excess kurtosis of the DM-SNR curve']  # Используем только эти столбцы

X = df[needed_columns]
y = df['target_class']

print(X.shape, y.shape)

## 3. Графики

Проведем визуальный анализ данных. Построим оценки плотности по каждому признаку отдельно для каждого класса. Какие признаки лучше всего разделяют классы? Как это отразится на результатах модели?

In [ ]:
sns.set_theme(font_scale=0.8)

plot = sns.PairGrid(data, x_vars=needed_columns, y_vars=needed_columns, hue="target_class", diag_sharey=False)

plot.map_diag(sns.kdeplot)
plot.map_lower(sns.scatterplot)
plot.add_legend()

**Ответ:**

Хорошо разделяются те признаки, которые визально различиысы на данных гафиках.

### 4. Разделим датасет и трансформируем данные

Добавьте  свободный член, присоединив столбец из единиц к `X`. Случайно разделите выборку на обучающую и тестовую части в соотношении 4:1.

In [ ]:
X["intercept"] =  np.ones((X.shape[0], 1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

У признаков должен быть одинаковый масштаб. Преобразуйте датасет соответствующим образом.

In [ ]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### 5. Обучим модель

Создайте и обучите логистическую регрессию. `fit_intercept` укажите `True`.

In [ ]:
model = LogisticRegression(fit_intercept = False)

model.fit(X_train, y_train)

Напечатайте коэффициенты модели. О чём говорят знаки `+` и `-`? Почему коэффициент, отвечающий за единичный столбец, равен нулю?

In [ ]:
model.coef_, model.intercept_

**Выводы:**

Знаки + и - перед коэффициентами указывают на направление влияния признаков на целевую переменную.

### 6. Проверим качество модели

Для  объектов из тестовой выборки оцените вероятность того, что это пульсар, при помощи метода [`predict_proba`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). Выведите эти значения, а также итоговые предсказания модели методом `predict`

In [ ]:
prob = model.predict_proba(X_test)

print("Вероятности принадлежности к классам:")
print(prob)

y_pred = model.predict(X_test)

# Вывод итоговых предсказаний
print("Итоговые предсказания:")
print(y_pred)

Теперь проверим качество нашей модели. Посчитайте его на тестовой выборке при помощи метрики Accuracy, показывающей долю правильных предсказаний:
$$
    Accuracy(y, \widehat{y}) = \frac{1}{n}\sum^{n-1}_{i = 0} I\{\widehat{y_i} = y_i\}
$$

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)

print(f"Точность модели на тестовой выборке: {accuracy:.2f}")

Другими важными метриками для оценки качества модели являются precision, recall и F1. Precision показывает точность нашей модели, то есть долю правильно классифицированных объектов из всех, которым был присвоен первый класс. Recall - это полнота модели, то есть доля правильно классифицированных объектов из всех. F1 суммирует результаты этих двух метрик, считая их среднее гармоническое. Чуть подробнее можно прочитать про эти величины в [статье](https://habr.com/ru/articles/661119/).

<img width=450 src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/800px-Precisionrecall.svg.png">

Посчитайте все три метрики для вашей модели. Сделайте выводы о качестве предсказаний.

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

**Выводы:**

Precision: 77% объектов, которым модель присвоила положительный класс, действительно принадлежат к этому классу.

Recall: Модель успешно обнаружила 83% объектов, которые действительно принадлежат к положительному классу.

F1 Score - комбинированная метрика, учитывает как точность, так и полноту. Значение 0.80 говорит о том, что модель достаточно хорошо сбалансирована.